## RNA Velocity using scVelo
Marissa Esteban

Data: CITEseq YS006_UW

I did pre processing in Suerat and saved object as .h5Seurat to perform RNAvelo in Python wtih scVelo

In [1]:
# Setup
import scanpy as sc
import scvelo as scvelo
import anndata
import loompy
import mygene

/Users/marissaestaban/miniforge3/lib/python3.12/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# loading in Seurat object and loading in data

adata_import = sc.read_h5ad("/Users/marissaestaban/Documents/CITEseq_data/ys006_named.h5ad")
ldata_import = sc.read_loom('/Users/marissaestaban/SRSP Laboratory Dropbox/SRSP Lab/Resources/Town Square - Data Share/Sequencing Repository/Data/CITEseq Data/YS006_Splicing-Alignment/possorted_genome_bam_751VB.loom')

ldata = ldata_import
adata = adata_import

# match CELL names
ldata.obs.index = [x.split(':')[1] for x in ldata.obs.index]    # possorted_genome_bam_751VB:AAAGTCCGTAAGGTCGx
ldata.obs.index = [x[:-1] for x in ldata.obs.index] 
adata.obs.index = [x.split('-')[0] for x in adata.obs.index]    # TTTGGAGTCGGTTGTA-1_1

# subset loom to only cells in adata
ldata = ldata[adata.obs.index, :]

In [3]:
print("=== Seurat-derived AnnData (adata) ===")
print("Cells (obs):", adata.n_obs)
print("Genes (var):", adata.n_vars)
print("\nFirst 10 cell names:", adata.obs.index[:10].tolist())
print("\nFirst 10 gene names:", adata.var.index[:10].tolist())

print("\n\n=== Loom RNA velocity AnnData (ldata) ===")
print("Cells (obs):", ldata.n_obs)
print("Genes (var):", ldata.n_vars)
print("\nFirst 10 cell names:", ldata.obs.index[:10].tolist())
print("\nFirst 10 gene names:", ldata.var.index[:10].tolist())

print("\n\n=== Layer shapes in ldata ===")
for layer in ["spliced", "unspliced", "ambiguous"]:
    if layer in ldata.layers:
        print(f"{layer}: {ldata.layers[layer].shape}")


=== Seurat-derived AnnData (adata) ===
Cells (obs): 4207
Genes (var): 19581

First 10 cell names: ['AAACCCAAGGGATGTC', 'AAACCCAAGGTACCTT', 'AAACGAAAGGAATTAC', 'AAACGAAAGTGCACAG', 'AAACGAAGTTGCCATA', 'AAACGAATCCAAGGGA', 'AAACGCTAGGGCATGT', 'AAACGCTAGTACTGTC', 'AAACGCTAGTCCTACA', 'AAACGCTCATGAATCC']

First 10 gene names: ['Xkr4', 'Gm1992', 'Gm19938', 'Rp1', 'Sox17', 'Gm37587', 'Mrpl15', 'Lypla1', 'Tcea1', 'Rgs20']


=== Loom RNA velocity AnnData (ldata) ===
Cells (obs): 4207
Genes (var): 33696

First 10 cell names: ['AAACCCAAGGGATGTC', 'AAACCCAAGGTACCTT', 'AAACGAAAGGAATTAC', 'AAACGAAAGTGCACAG', 'AAACGAAGTTGCCATA', 'AAACGAATCCAAGGGA', 'AAACGCTAGGGCATGT', 'AAACGCTAGTACTGTC', 'AAACGCTAGTCCTACA', 'AAACGCTCATGAATCC']

First 10 gene names: [np.str_('ENSMUSG00000079800'), np.str_('ENSMUSG00000095092'), np.str_('ENSMUSG00000079794'), np.str_('ENSMUSG00000079192'), np.str_('ENSMUSG00000094799'), np.str_('ENSMUSG00000095250'), np.str_('ENSMUSG00000095787'), np.str_('ENSMUSG00000095672'), np.str_('

In [4]:
# converting the loom's ENSEMBL ID's to match 

mg = mygene.MyGeneInfo()
gene_index = ldata.var.index.to_series()

# only grab the genes that have ENSEMBL IDs
is_ensembl = gene_index.str.startswith("ENSMUSG")

print("Total genes:", gene_index.shape[0])
print("ENSEMBL-like genes:", is_ensembl.sum())
print("Already-symbol-like genes:", (~is_ensembl).sum(), '\n')

# only query the ENSMUSG IDs
ens_ids = gene_index[is_ensembl].unique().tolist()

results = mg.querymany(
    ens_ids,
    scopes="ensemblgene",   # or "ensembl.gene" depending on mygene version, but this usually works
    fields="symbol",
    species="mouse",
    as_dataframe=False
)

# Build mapping dict ENSMUSG -> symbol
ens_to_symbol = {r["query"]: r.get("symbol", None) for r in results}

# QC on mapping
total_ens = len(ens_ids)
mapped_ens = sum(v is not None for v in ens_to_symbol.values())
print("Total ENSMUSG IDs:", total_ens)
print("Mapped ENSMUSG → symbol:", mapped_ens)
print("Unmapped ENSMUSG:", total_ens - mapped_ens)

# 4) Create a 'symbol' column:
# default: keep existing name
ldata.var["symbol"] = gene_index.copy()

# replace ENSMUSG entries with their symbol
for g in gene_index[is_ensembl]:
    sym = ens_to_symbol.get(g, None)
    if sym is not None:
        ldata.var.at[g, "symbol"] = sym


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


Total genes: 33696
ENSEMBL-like genes: 248
Already-symbol-like genes: 33448 



13 input query terms found no hit:	['ENSMUSG00000095742', 'ENSMUSG00000095728', 'ENSMUSG00000095076', 'ENSMUSG00000121317', 'ENSMUSG000


Total ENSMUSG IDs: 248
Mapped ENSMUSG → symbol: 203
Unmapped ENSMUSG: 45


In [14]:
gene_index = ldata.var.index.to_series()
is_ensembl = gene_index.str.startswith("ENSMUSG")

print("Total ENSMUSG IDs after mapping: ", is_ensembl.sum())

Total ENSMUSG IDs after mapping:  45


In [26]:
# Intersect with Seurat genes
# First, make the symbol column the new index for ldata
ldata.var_names = ldata.var["symbol"].values
ldata.var_names_make_unique()  # This handles any duplicate gene symbol

print("Total number of var names: ", len(ldata.var_names))

# Now find common genes between the two datasets
common_genes = adata.var.index.intersection(ldata.var.index)
print("Overlapping genes after symbol mapping:", len(common_genes))

# Subset both datasets to common genes
adata_subset = adata[:, common_genes].copy()
ldata_subset = ldata[:, common_genes].copy()

print(f"adata shape: {adata_subset.shape}")
print(f"ldata shape: {ldata_subset.shape}")

Total number of var names:  33696
Overlapping genes after symbol mapping: 19532
adata shape: (4207, 19532)
ldata shape: (4207, 19532)


In [30]:
# Sanity Checks

# Verify cell barcodes match
print("Cells match:", (adata_subset.obs.index == ldata_subset.obs.index).all())

# If they don't match, reorder ldata_subset to match adata_subset:
ldata_subset = ldata_subset[adata_subset.obs.index, :].copy()

# Double-check genes are in same order
print("Genes match:", (adata_subset.var.index == ldata_subset.var.index).all())

Cells match: True
Genes match: True


In [31]:
# Add velocity layers to adata_subset
adata_subset.layers["spliced"] = ldata_subset.layers["spliced"].copy()
adata_subset.layers["unspliced"] = ldata_subset.layers["unspliced"].copy()
adata_subset.layers["ambiguous"] = ldata_subset.layers["ambiguous"].copy()

# Verify they were added
print("Layers in adata_subset:", list(adata_subset.layers.keys()))
print("Spliced shape:", adata_subset.layers["spliced"].shape)
print("Unspliced shape:", adata_subset.layers["unspliced"].shape)

Layers in adata_subset: ['spliced', 'unspliced', 'ambiguous']
Spliced shape: (4207, 19532)
Unspliced shape: (4207, 19532)
